In [88]:
import matplotlib.pyplot as plt
import math
import pandas as pd
import numpy as np
from tqdm import tqdm
from anastruct import SystemElements

In [89]:
qntd_interable = 5
P1 = 2
P2 = 5
L = 1.2

In [91]:
def create_truss (cncts, P1, P2, nodes):
    
    #Create the truss elements 
    for connection in cncts:
        element_truss = [list(nodes.get(connection[0])), list(nodes.get(connection[1]))]
        ss.add_truss_element(location = element_truss)

    #Include in the struct the extern loads 
    loads = {'B': (0, -P1, 0), 'C': (0, -P2, 0)}
    for load in loads:
        Fx, Fy, _ = loads.get(load)
        node_id = ss.find_node_id(nodes.get(load))
        ss.point_load(node_id=node_id, Fx=Fx, Fy=Fy)

    #Extern supports 
    supports = {'A': (1, 1, 0), 'F': (1, 1, 0)}
    for support in supports:
        coord = supports.get(support)
        for l in range(len(coord)):
            if coord[l] != 0:
                node_id = ss.find_node_id(nodes.get(support))
                dir = l+1
                ss.add_support_roll(node_id=node_id, direction=dir)

    ss.solve()

    elements = ss.get_element_results()
    length = []
    for element in elements:
        if element['N'] <= 3.5:
            length.append(element['length'])
        else:
            length.append(np.nan)
    
    return min(length)

In [92]:
dict_length = {}

for i in tqdm(range(qntd_interable), leave=True, position=0):
    X = np.linspace(-0.5, +0.5, 100)
    length_interval = []
    for value in X:
        
        x0, x1, x2, x3, x4 = 0, 0, 0, 0, 0
        if i==0:
            x0 = value
        if i==1:
            x1 = value
        if  i==2:
            x2 = value
        if i==3:
            x3 = value
        if i==4:
            x4 = value
        
        ss = SystemElements()
        
        nodes = {
        'A': (0, 0), 
        'B': (L, 0),
        'C': (2*L, x4), 
        'D': ((2*L) + x2, L+x3),
        'E': (L+x0, L+x1),
        'F': (0, L)
        }

        cncts = [('A', 'B'), ('A', 'F'), ('B', 'C'),
                 ('B', 'E'), ('B', 'F'), ('C', 'D'),
                 ('C', 'E'), ('D', 'E'), ('E', 'F')]
        
        min_length = create_truss(cncts, P1, P2, nodes)
        length_interval.append(min_length)
    dict_length[i] = length_interval

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:01<00:00,  3.23it/s]


In [105]:
length_dataframe = pd.DataFrame(dict_length)
display(length_dataframe)

,0,1,2,3,4
0,1.200000,0.700000,0.700000,0.700000,1.200000
1,1.200000,0.710101,0.710101,0.710101,1.200000
2,1.200000,0.720202,0.720202,0.720202,1.200000
3,1.200000,0.730303,0.730303,0.730303,1.200000
4,1.200000,0.740404,0.740404,0.740404,1.200000
...,...,...,...,...,...
95,0.740404,1.200000,1.200000,1.200000,0.740404
96,0.730303,1.200000,1.200000,1.200000,0.730303
97,0.720202,1.200000,1.200000,1.200000,0.720202
98,0.710101,1.200000,1.200000,1.200000,0.710101


In [107]:
length_dataframe.columns = ['x0', 'x1', 'x2', 'x3', 'x4']

In [108]:
length_dataframe.head(3)

,x0,x1,x2,x3,x4
0,1.2,0.700000,0.700000,0.700000,1.2
1,1.2,0.710101,0.710101,0.710101,1.2
2,1.2,0.720202,0.720202,0.720202,1.2


In [109]:
length_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   x0      100 non-null    float64
 1   x1      100 non-null    float64
 2   x2      100 non-null    float64
 3   x3      100 non-null    float64
 4   x4      100 non-null    float64
dtypes: float64(5)
memory usage: 4.0 KB
